# 카카오 키워드 & Map API
- 대표 컬럼: [id, place_name, keyword, category_group_name, x, y, base_url, rating]

In [63]:
import requests
import pandas as pd
import numpy as np
import folium
from folium.plugins import MiniMap

In [64]:
# import requests 
# url = 'https://dapi.kakao.com/v2/local/search/keyword.json' 
# params = {'query': "성산일출봉 전기충전소",'page': 1} 
# headers = {"Authorization": "KakaoAK 1c259a04720f7a1bf08122bd44214fa7"}
# places = requests.get(url, params=params, headers=headers).json()['documents']
# total = requests.get(url, params=params, headers=headers).json()['meta']['total_count'] 
# if total > 45: 
#     print(total,'개 중 45개 데이터밖에 가져오지 못했습니다!') 
# else : 
#     print('모든 데이터를 가져왔습니다!') 
#     print(places)

In [65]:
##카카오 API
## region에는 '성산일출봉 전기충전소' 검색명이 들어갈 것임
## page_num은 1~3이 입력될 건데, 한 페이지 당 검색목록이 최대 15개임.
## 만약 page_num이 4이상이 되면 3페이지랑 같은 15개의 결과 값을 가져옴. 그래서 1~3만 쓰는 것임
## 입력 예시 -->> headers = {"Authorization": "KakaoAK f221u3894o198123r9"}
## ['meta']['total_count']은 내가 '성산일출봉 전기충전소'를 검색했을 때, 나오는 총 결과 값. 
## ['meta']['total_count']이 45보다 크면 45개만 가져오게 됨


def elec_location(region,page_num):
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
    params = {'query': region,'page': page_num}
    headers = {"Authorization": "KakaoAK 1c259a04720f7a1bf08122bd44214fa7"}

    places = requests.get(url, params=params, headers=headers).json()['documents']
    total = requests.get(url, params=params, headers=headers).json()['meta']['total_count']
    if total > 45:
        print(total,'개 중 45개 데이터밖에 가져오지 못했습니다!')
    else :
        print('모든 데이터를 가져왔습니다!')
    return places

In [78]:
## 이 함수는 위 함수 결과 값(1 ~ 45개) 하나하나 분리해서 저장할 것임
## 1번 결과 값 안에는 1번 충전소 이름, 위도, 경도, 전화번호, 도로명 주소 등이 있는데 각각 배열에 저장
## 우리는 충전소 ID, 충전소 이름, 위도, 경도, 도로명주소, 사이트주소를 저장할 것임

def elec_info(places):
    X = []
    Y = []
    stores = []
    road_address = []
    place_url = []
    ID = []
    ratings = []
    for place in places:
        X.append(float(place['x']))
        Y.append(float(place['y']))
        stores.append(place['place_name'])
        road_address.append(place['road_address_name'])
        place_url.append(place['place_url'])
        ID.append(place['id'])
        # ratings.append(float(place['rating']))

    ar = np.array([ID,stores, X, Y, road_address,place_url]).T
    df = pd.DataFrame(ar, columns = ['ID','stores', 'X', 'Y','road_address','place_url'])
    return df

In [79]:
## 여러개의 키워드를 검색할 때 사용할 함수임
## location_name에는 ['성산일출봉 전기충전소, '한림공원 전기충전소', ... ,'모슬포 전기충전소']처럼 배열이 입력

def keywords(location_name):
    df = None
    for loca in location_name:
        for page in range(1,4):
            local_name = elec_location(loca, page)
            local_elec_info = elec_info(local_name)

            if df is None:
                df = local_elec_info
            elif local_elec_info is None:
                continue
            else:
                df = pd.concat([df, local_elec_info],join='outer', ignore_index = True)
    return df

In [80]:
# 지도로 보여주기
# [id, place_name, keyword, category_group_name, x, y, base_url, rating]

def make_map(dfs):
    # 지도 생성하기
    m = folium.Map(location=[33.4935,126.6266],   # 기준좌표: 제주어딘가로 내가 대충 설정
                   zoom_start=12)

    # 미니맵 추가하기
    minimap = MiniMap() 
    m.add_child(minimap)

    # 마커 추가하기
    for i in range(len(dfs)):
        
        folium.Marker([dfs['Y'][i],dfs['X'][i]],
                  tooltip=dfs['stores'][i],
                  popup=dfs['place_url'][i],
                  ).add_to(m)
        
    return m

In [84]:
## 여기 두 개 키워드처럼 가까운 거리에 있는 키워드를 입력하면 
## 중복해서 전기충전소를 검색할 가능성이 아주 놓기 때문에
## drop_duplicates를 해주고 인덱스 리셋을 해준다

location = ['서귀동 관광지','서귀동 맛집']
df = keywords(location)
df = df.drop_duplicates('ID')
df = df.reset_index()
make_map(df)

253 개 중 45개 데이터밖에 가져오지 못했습니다!
253 개 중 45개 데이터밖에 가져오지 못했습니다!
253 개 중 45개 데이터밖에 가져오지 못했습니다!
2537 개 중 45개 데이터밖에 가져오지 못했습니다!
2537 개 중 45개 데이터밖에 가져오지 못했습니다!
2537 개 중 45개 데이터밖에 가져오지 못했습니다!


In [82]:
df

,index,ID,stores,X,Y,road_address,place_url
0,0,11635431,올레길 7코스(서귀포-월평 올레),126.51530966206293,33.23738579332568,제주특별자치도 서귀포시 중정로 22,http://place.map.kakao.com/11635431
1,1,10626234,올레길 1코스(시흥-광치기 올레),126.90636648646604,33.47866074455451,,http://place.map.kakao.com/10626234
2,2,8015963,올레길 6코스(쇠소깍-서귀포 올레),126.59230607000688,33.24127212174994,,http://place.map.kakao.com/8015963
3,3,13597626,올레길 18코스(제주원도심-조천 올레),126.58214303914238,33.52522810308715,,http://place.map.kakao.com/13597626
4,4,12753473,올레길 17코스(광령-제주원도심 올레),126.4726047828175,33.50563877194622,,http://place.map.kakao.com/12753473
5,5,12753454,올레길 13코스(용수-저지 올레),126.21230336658091,33.328625315540165,,http://place.map.kakao.com/12753454
6,6,11943943,올레길 16코스(고내-광령 올레),126.38979561557086,33.46649759191228,,http://place.map.kakao.com/11943943
7,7,11828668,올레길 5코스(남원-쇠소깍 올레),126.66743988738492,33.27078076230098,,http://place.map.kakao.com/11828668
8,8,12753435,올레길 10코스(화순-모슬포 올레),126.29320987840607,33.216890444802004,,http://place.map.kakao.com/12753435
9,9,17605888,올레길 20코스(김녕-하도 올레),126.80547165134814,33.55424539993182,,http://place.map.kakao.com/17605888
